# Example of Sensitivity Analysis

In [ ]:
# Defaults
n_samples = 1024 * 21

In [ ]:
from itertools import repeat

import numpy as np
import pandas as pd

import dask.bag as db
from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from SALib.sample import latin
from SALib.analyze import delta as delta_mim

from mhpc_project.parameters import ParametersTable
from mhpc_project.calibration import Loss
from mhpc_project.models import SMC50
from mhpc_project.measures import KGE
from mhpc_project.utils import date_parser

In [ ]:
observations = pd.read_csv('../data/Matsch B2/obs.csv',
                           na_values=['-9999', '-99.99'],
                           usecols=[0, 7],
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0,
                           squeeze=True)
observations.index.rename('datetime', inplace=True)
model = SMC50('../data/Matsch B2/geotop', timeout=120)
parameters = ParametersTable.read_csv('../data/Calibration Parameters/testbed.csv')
measure = KGE(observations)
loss = Loss(model, parameters, measure)

In [ ]:
cluster = Cluster()
client = Client(cluster)

In [ ]:
problem = {'num_vars': parameters.num_vars,
           'names': parameters.names,
           'bounds': list(repeat((0.0, 1.0), parameters.num_vars))}

samples = latin.sample(problem, n_samples)

In [ ]:
bag = db.from_sequence(samples, npartitions=512).map(loss)
losses = bag.compute()

In [ ]:
SA = delta_mim.analyze(problem, samples, losses)
SA.to_df().sort_values('delta', key=np.abs, ascending=False)